In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline

from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [ ]:
!unzip '../input/dogs-vs-cats/train.zip' -d '/tmp/cats-vs-dogs'
!unzip '../input/dogs-vs-cats/test1.zip' -d '/tmp/cats-vs-dogs'

In [ ]:
base = '/tmp/cats-vs-dogs/'
train_loc = base + 'train/'
test_loc = base + 'test1/'

In [ ]:
train_files = os.listdir(train_loc)
categories = []
for file in train_files:
    val = file.split('.')[0]
    if(val == 'dog'):
        categories.append(1)
    else:
        categories.append(0)
        


In [ ]:
df = pd.DataFrame({
    'files' : train_files,
    'class': categories
})

In [ ]:
df.head()

In [ ]:
fig=plt.figure(figsize=(16, 16))
columns = 4
rows = 5
i=1
for filename in os.listdir(train_loc)[0:20]:
    image = mpl.image.imread(train_loc + filename)
    fig.add_subplot(rows, columns, i)
    plt.imshow(image)
    i+=1
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def read_images_from_dataframe(df, IMAGE_DIR, file_col='files', class_col='class', class_mode='raw',
                               target_size=(224, 224), batch_size=32, shuffle=True, validation_split=0.2):
    img_datagen = ImageDataGenerator(rescale=1. / 255,
                                     validation_split=validation_split)

    train_generator = img_datagen.flow_from_dataframe(df,
                                                      directory=IMAGE_DIR,
                                                      x_col=file_col,
                                                      y_col=class_col,
                                                      target_size=target_size,
                                                      batch_size=batch_size,
                                                      shuffle=shuffle,
                                                      class_mode=class_mode,
                                                      subset='training')

    test_generator = img_datagen.flow_from_dataframe(df,
                                                     directory=IMAGE_DIR,
                                                     x_col=file_col,
                                                     y_col=class_col,
                                                     target_size=target_size,
                                                     batch_size=batch_size,
                                                     shuffle=shuffle,
                                                     class_mode=class_mode,
                                                     subset='validation')

    return train_generator, test_generator


In [ ]:
tr_gen, test_gen = read_images_from_dataframe(df, train_loc)

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input

model = keras.Sequential()

model.add(keras.applications.resnet50.ResNet50(weights = 'imagenet', include_top=False))

model.add(keras.layers.GlobalAveragePooling2D())

model.add(keras.layers.Dense(64, activation="relu"))

model.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=3e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
for layer in model.layers[0].layers:
    if layer.name == 'conv5_block1_0_conv':
        break
    layer.trainable=False

In [ ]:
history = model.fit(tr_gen, validation_data=test_gen, epochs=5, 
                    steps_per_epoch=tr_gen.n//tr_gen.batch_size,
                   validation_steps=test_gen.n//test_gen.batch_size)

In [ ]:
for index, layer in enumerate(base_model.layers):
    print(index, layer.name)

In [ ]:
tr_gen.n

In [ ]:
for layer in base_model.layers[0].layers:
    if layer.name == 'conv5_block1_0_conv':
        break
    layer.trainable=False




In [ ]:
print("hi")

In [ ]:
model

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 5, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['acc'], color='b', label="Training accuracy")
ax2.plot(history.history['val_acc'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 5, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()


In [ ]:
test_filenames = os.listdir(test_loc)

In [ ]:
test_df = pd.DataFrame({
    'files' : test_filenames
})

In [ ]:
test_df.head()

In [ ]:
off_test_gen = ImageDataGenerator(rescale=1./255)

off_test_generator = off_test_gen.flow_from_dataframe(test_df, test_loc, x_col="files", y_col=None, class_mode=None, batch_size=32, target_size=(224, 224), shuffle=False)

In [ ]:
tsamples = test_df.shape[0]

In [ ]:
predictions = model.predict_generator(off_test_generator, steps=np.ceil(tsamples/32))

In [ ]:
threshold = 0.5
test_df['category'] = np.where(predictions > threshold, 1,0)

In [ ]:
test_df

In [ ]:
ans = test_df.copy()

In [ ]:
ans["label"] = ans["category"]

In [ ]:
ans["id"] = range(1, 12501)

In [ ]:
ans.head()

In [ ]:
ans = ans.drop(['files', 'category'], axis=1)

In [ ]:
ans[ans.columns[::-1]]

In [ ]:
ans.to_csv("sub.csv", index=False)